Imports

In [328]:
import pygad
import random
from read_data import read_dataset_1, translate_1, read_dataset_3, translate_3, translate_1_testing
from models import SimulationEnvironment
from optimizer_components import map_index_to_operation
import plotly.figure_factory as ff

Parameters

In [329]:
order_amount = 5
earliest_time_slot = 0
last_time_slot = 5000 # shouldn't actually be necessary for this

Input

In [330]:
input, orders, instance = read_dataset_1(use_instance=6, order_amount=order_amount, earliest_time=earliest_time_slot, last_time=last_time_slot)
#recipes, workstations, resources, tasks, orders_model = translate_1(instance, orders, earliest_time_slot, last_time_slot)
recipes, workstations, resources, tasks, orders_model = translate_1_testing(instance, orders, earliest_time_slot, last_time_slot)
env = SimulationEnvironment(workstations, tasks, resources, recipes)

Setup

In [331]:
assignments = []
start_times = []

duration_lookup_table = dict()
for task in tasks:
    if not task.external_id in duration_lookup_table:
            duration_lookup_table[task.external_id] = dict()
    for workstation in env.get_valid_workstations(task.external_id):
        duration_lookup_table[task.external_id][workstation.id] = env.get_duration(task.external_id, workstation.id)

operations = []
order_for_index = []
for order in orders_model:
    for resource in order.resources:
        recipe = resource.recipes[0] # just use recipe 0 for now
        recipe_tasks = env.get_all_tasks_for_recipe(recipe.id)
        results = dict()
        for task in recipe_tasks:
            if task.result_resources[0][0] not in results:
                results[task.result_resources[0][0]] = []
            results[task.result_resources[0][0]].append(task)
        for key in results:
            #operations.append(random.choice(results[key]).id)
            operations.append(random.choice(results[key]).external_id)
            order_for_index.append(order.id)

for operation in operations:
    workstation = env.get_valid_workstations(operation)
    # random init
    assignments.append(random.choice(workstation).id)
    assignments.append(0) # start time slot

In [332]:
print(duration_lookup_table)
print(operations)

{0: {1: 108, 2: 119}, 1: {4: 136}, 2: {4: 150, 5: 155}, 3: {5: 214, 3: 144}, 4: {3: 44}, 5: {3: 65}, 6: {3: 87}, 7: {3: 70}, 8: {3: 120}}
[6, 7, 8, 0, 1, 2, 6, 7, 8, 0, 1, 2, 6, 7, 8]


Helper Functions

In [333]:
def get_order_for_index(index):
    order = orders[0]
    current_order = orders[0]
    order_id = 0
    jobs = instance[1] # contains amount of jobs for each order
    i = 0
    while i < index:
    #for i in range(0, index, 2): # encoding
        if i+len(jobs[current_order[0]]) > index: #found correct order
            break
        i += len(jobs[current_order[0]]) * 2
        order_id += 1
        if order_id < len(orders):
            current_order = orders[order_id]
    return order

def get_prev_scheduled_operations(individual, index): # TODO: double check orders
    # return assignments + start times as list of tuples
    #id, target_order = map_index_to_operation(int(index/2), orders, env)
    target_order = get_order_for_index(index)
    jobs = instance[1]
    idx = 0
    prev_scheduled = []
    # find index of task in specific order
    order_start_index = 0
    i = 0
    while i < index:
    #for i in range(index):
        #curr_id, curr_order = map_index_to_operation(int(i/2), orders, env)
        curr_order = get_order_for_index(index)
        if curr_order[2] == target_order[2]: # assuming 2 is id
            order_start_index = i
            break
        i += len(jobs[curr_order[0]]) * 2 # skip the rest of the order
    if index - order_start_index > 0:
        prev_scheduled.append((individual[index - 2], individual[index -1], duration_lookup_table[operations[int((index-2)/2)]][individual[index - 2]])) # actually only need previous, since tasks in same order are sorted        
    return prev_scheduled

def calculate_start_time(individual):
    # duration_lookup_table[operations[int((idx-1)/2)]][solution[idx-1]]
    i = 0
    operation_index = 0
    for idx in range(0, len(individual), 2):
    #for gene in individual:
        gene = individual[idx]
        on_workstation = (0, 0) # find all assignments to the same workstation
        prev_operations = [] # find all operations belonging to the same order, which need to be scheduled before the current operation
        # calculate start time, aka choose max(last end time on workstation, last end time of previous sequenced operation for task)
        k = 0
        for j in range(0,idx,2): # introduces dependency on sequence of the orders
            if individual[j] == gene: # same workstation assigned
                if individual[j+1] + duration_lookup_table[operations[int(j/2)]][individual[j]] > on_workstation[0] + on_workstation[1]:
                    on_workstation = (individual[j+1], duration_lookup_table[operations[int(j/2)]][individual[j]])
        prev_operations = get_prev_scheduled_operations(individual, idx)
        if prev_operations:
            prev_operation = max(prev_operations, key=lambda tup: tup[1] + tup[2])
        else:
            prev_operation = (0, 0, 0)
        individual[idx+1] = max(on_workstation[0] + on_workstation[1], prev_operation[1] + prev_operation[2])
        operation_index += 1
    return individual

In [334]:
def get_colors(n): 
    ret = [] 
    r = int(random.random() * 256) 
    g = int(random.random() * 256) 
    b = int(random.random() * 256) 
    step = 256 / n 
    for i in range(n): 
        r += step 
        g += step 
        b += step 
        r = int(r) % 256 
        g = int(g) % 256 
        b = int(b) % 256 
        ret.append((r,g,b))  
    return ret

def visualize(data):
    # data format: 0 - workstation, 1 - job id, 2 - start time, 3 - duration
    colors = {}
    rgb_values = get_colors(len(orders))
    for i in range(len(orders)):
        colors[str(f'Order {i}')] = f'rgb({rgb_values[i][0]}, {rgb_values[i][1]}, {rgb_values[i][2]})' # just ignore colors for now
    composed_data = []
    
    for i in range(len(data)):
        label = f'W{data[i][0]}'
        start = data[i][2]
        end = start + data[i][3]
        composed_data.append(
                    dict(Task=label, Start=start, Finish=end, Resource=f'Order {order_for_index[i]}')
                )
        #print(composed_data)
    fig = ff.create_gantt(composed_data, colors=colors, index_col='Resource', show_colorbar=True,
                        group_tasks=True, showgrid_x=True)
    fig.update_layout(xaxis_type='linear')
    """import plotly.express as px
    fig = px.timeline(composed_data, x_start='Start', x_end='Finish', y='Task', color='Resource')""" # for some reason doesn't do what it's supposed to do
    fig.show()

GA Functions

In [335]:
def mutation_function(offspring, ga_instance):
    i = 0
    operation_index = 0
    p = 0.1#ga_instance.mutation_percent_genes
    idx = 0
    for gene in offspring:
        if i == 0:
            if random.random() < p:
                # mutate
                # according to paper, calc workload of each elligible workstation, switch to lowest workload
                workload = []
                valid_workstations = env.get_valid_workstations(operations[int(idx/2)]) # TODO: double check
                for j in range(len(valid_workstations)): 
                    count = 0
                    for k in range(0, gene, 2):
                        if offspring[k] == valid_workstations[j].id:
                            count += 1
                    workload.append((valid_workstations[j], count))
                current_count = 0
                for t in workload:
                    if t[0] == gene:
                        current_count = t[1]
                        break
                # gather all workloads smaller than the current workstations workload
                lower_loads = []
                smallest_load = t
                for t in workload:
                    if t[0] != gene and t[1] < current_count:
                        lower_loads.append(t)
                        if t[1] < smallest_load[1]:
                            smallest_load = t
                #offspring[idx] = random.choice(lower_loads)[0] # choose random smaller workstation
                offspring[idx] = smallest_load[0].id # just use smallest workload possible
        i+=1
        if i > 1:
            i = 0
            operation_index += 1
        idx += 1
    # re-calculate start times
    calculate_start_time(offspring)
    return offspring

# make sure crossover is performed at workstation assignments
def crossover_function(parents, offspring_size, ga_instance):
    split_point = random.randint(0, len(assignments))
    if split_point % 2 == 1:
        split_point -= 1
    parent1 = parents[0].copy()
    parent2 = parents[1].copy()
    offspring = []
    for i in range(len(assignments)):
        if i < split_point:
            offspring.append(parent1[i])
        else:
            offspring.append(parent2[i])
    return offspring

def fitness_function(solution, solution_idx):
    fitness = 1
    i = 0
    operation_index = 0
    max = -float('inf')
    min = float('inf')
    for idx in range(1, len(solution), 2):
        start = solution[idx]
        end = start + duration_lookup_table[operations[int((idx-1)/2)]][solution[idx-1]] # double check operations
        if start < min:
            min = start
        if end > max:
            max = end
        operation_index += 1
    fitness = abs(max - min)
    return -fitness


Run

In [336]:
num_genes = len(assignments)
num_generations = 5000
num_parents_mating = 50
sol_per_pop = 100
init_range_low = 0
init_range_high = last_time_slot
parent_selection_type = 'rws'
keep_parents = 10
crossover_type = crossover_function
mutation_type = mutation_function
mutation_percentage_genes = 10#0.1
fitness_func = fitness_function
gene_type = int
space_workstations = {'low': 0, 'high': len(workstations)-1} #0?
space_time = {'low': 0, 'high': last_time_slot}
gene_space = []
i = 0
for j in range(len(assignments)):
    if i == 0:
        valid_workstations = env.get_valid_workstations(operations[int(j/2)])
        space = []
        for k in range(len(valid_workstations)):
            space.append(valid_workstations[k].id)
        gene_space.append(space)
    else:
        gene_space.append(space_time) # shouldn't be needed because of manual mutation and crossover
    i += 1
    if i > 1:
        i = 0
ga_instance = pygad.GA(num_generations=num_generations, num_parents_mating=num_parents_mating, fitness_func=fitness_func, sol_per_pop=sol_per_pop, num_genes=num_genes, init_range_low=init_range_low, init_range_high=init_range_high, parent_selection_type=parent_selection_type, keep_parents=keep_parents, crossover_type=crossover_type, mutation_type=mutation_type, mutation_percent_genes=mutation_percentage_genes, gene_type=gene_type, gene_space=gene_space)
ga_instance.run()
solution, solution_fitness, solution_idx = ga_instance.best_solution()

print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=abs(solution_fitness) - 1))

Parameters of the best solution : [   3    0    3   87    3  157    1  277    4  385    5  521    3  676
    3  763    3  833    2  953    4 1072    4 1208    3 1358    3 1445
    3 1515]
Fitness value of the best solution = 1634


In [337]:
data = []
for i in range(0, len(solution), 2):
    data.append([solution[i], operations[int(i/2)], solution[i+1], duration_lookup_table[operations[int(i/2)]][solution[i]]])
visualize(data)